<a href="https://colab.research.google.com/github/dolmani38/QA/blob/main/Korean_QA_on_Wiki_220224.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a QA System with BERT on Wikipedia

https://qa.fastforwardlabs.com/pytorch/hugging%20face/wikipedia/bert/transformers/2020/05/19/Getting_Started_with_QA.html

위의 내용을 한국어 QA로 변경

지식 base = wiki+네이버(view,kin,news)

In [1]:
!pip install wikipedia==1.4.0
!pip install sentence-transformers==1.1.1
!pip install transformers==4.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=78bc9af5887d743ce9f905a983c5747eb8a776ae2da91dfe810b1d2a44bbfd5a
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

##한국어 SQUAD 모델의 사용
https://huggingface.co/monologg/koelectra-base-v3-finetuned-korquad

##영어 + 한국어 STS 모델의 사용

#Extending Sentence Embeddings Models to New Languages

**Available Pre-trained Models**

*   **distiluse-base-multilingual-cased**: Supported languages: Arabic, Chinese, Dutch, English, French, German, Italian, Korean, Polish, Portuguese, Russian, Spanish, Turkish. Model is based on DistilBERT-multi-lingual.
*   **xlm-r-base-en-ko-nli-ststb**: Supported languages: English, Korean. Performance on Korean STSbenchmark: 81.47
*   **xlm-r-large-en-ko-nli-ststb**: Supported languages: English, Korean. Performance on Korean STSbenchmark: 84.05 --> 이거 사용!


참조:https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/multilingual-models.md



In [2]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

SQUAD_MODEL = "monologg/koelectra-base-v3-finetuned-korquad"
STS_MODEL = "xlm-r-large-en-ko-nli-ststb"

tokenizer = AutoTokenizer.from_pretrained(SQUAD_MODEL)
model = AutoModelForQuestionAnswering.from_pretrained(SQUAD_MODEL)
#reader = DocumentReader(SQUAD_MODEL)
embedder = SentenceTransformer(STS_MODEL)

  0%|          | 0.00/1.80G [00:00<?, ?B/s]

In [3]:
# Print iterations progress
class ProgressBar:

  def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '█', printEnd = "\r"):
    self.total = total
    self.prefix = prefix
    self.suffix = suffix
    self.decimals = decimals
    self.length = length
    self.fill = fill
    self.printEnd = printEnd
    self.ite = 0

  def printProgress(self,iteration, text):
      self.ite += iteration
      percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))

      filledLength = int(self.length * self.ite // self.total)
      bar = self.fill * filledLength + '-' * (self.length - filledLength)
      print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
      # Print New Line on Complete
      if self.ite == self.total:
          print()

In [4]:
class AnswerVoter:
  def __init__(self, threshold_score=3,max_rank=5):
    self.answes = {}
    self.threshold_score= threshold_score
    self.max_rank = max_rank

  def add_ans(self,ans,score,src,pb):
    key = ans.replace(' ','')
    if score > self.threshold_score:
      #print(' --- Candidate answer:',ans,score)
      pb.printProgress(0,'Candidate answer:' + str(ans) + ' ' + str(score))
      if key in self.answes:
        self.answes[key][1] += score
        if src in self.answes[key][2]:
          pass
        else:
          self.answes[key][2].append(src)
      else:
        self.answes[key] = [ans,score,[src]]

  def get_ans(self):
    answer = []
    sorted_x = sorted(self.answes.items(), key=lambda kv: kv[1][1],reverse=True)
    for i in range(min(self.max_rank,len(sorted_x))):
      answer.append(sorted_x[i])

    return answer

  def print(self):
    answer = self.get_ans()
    for ans in answer:
      print('Answer:',ans[1][0], ' score:',ans[1][1], ' source:',ans[1][2])
      #print('Answer:',ans)

In [5]:
import wikipedia as wiki
import pprint as pp
from collections import OrderedDict
import torch

class DocumentReader:
    def __init__(self, _tokenizer, _model):
        #self.READER_PATH = pretrained_model_name_or_path
        self.tokenizer = _tokenizer #AutoTokenizer.from_pretrained(self.READER_PATH)
        self.model = _model #AutoModelForQuestionAnswering.from_pretrained(self.READER_PATH)
        self.max_len = self.model.config.max_position_embeddings
        self.chunked = False

        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        #self.tokenizer.to(self.device)
        self.model.to(self.device)

    def tokenize(self, question, context):
        self.inputs = self.tokenizer.encode_plus(question, context, max_length=512, truncation=True, add_special_tokens=True, return_tensors="pt")
        self.inputs.to(self.device)
        self.input_ids = self.inputs["input_ids"].tolist()[0]

        if len(self.input_ids) > self.max_len:
            self.inputs = self.chunkify()
            self.chunked = True
            print('input_ids:',len(self.input_ids),'max_len:',self.max_len)

    def chunkify(self):
        """
        Break up a long article into chunks that fit within the max token
        requirement for that Transformer model.

        Calls to BERT / RoBERTa / ALBERT require the following format:
        [CLS] question tokens [SEP] context tokens [SEP].
        """

        # create question mask based on token_type_ids
        # value is 0 for question tokens, 1 for context tokens
        qmask = self.inputs['token_type_ids'].lt(1)
        qt = torch.masked_select(self.inputs['input_ids'], qmask)
        chunk_size = self.max_len - qt.size()[0] - 1 # the "-1" accounts for
        # having to add an ending [SEP] token to the end

        # create a dict of dicts; each sub-dict mimics the structure of pre-chunked model input
        chunked_input = OrderedDict()
        for k,v in self.inputs.items():
            q = torch.masked_select(v, qmask)
            c = torch.masked_select(v, ~qmask)
            chunks = torch.split(c, chunk_size)

            for i, chunk in enumerate(chunks):
                if i not in chunked_input:
                    chunked_input[i] = {}

                thing = torch.cat((q, chunk))
                if i != len(chunks)-1:
                    if k == 'input_ids':
                        thing = torch.cat((thing, torch.tensor([102])))
                    else:
                        thing = torch.cat((thing, torch.tensor([1])))

                chunked_input[i][k] = torch.unsqueeze(thing, dim=0)
        return chunked_input.to(self.device)

    def get_answer(self,answer,src,pb):
        if self.chunked:

            for k, chunk in self.inputs.items():
                #chunk.to(self.device)
                outputs = self.model(**chunk)

                answer_start = torch.argmax(outputs[0])
                answer_end = torch.argmax(outputs[1]) + 1
                ans = self.convert_ids_to_string(chunk['input_ids'][0][answer_start:answer_end])
                if ans.startswith(('[CLS]','[SEP]',' ','°')) or ans=='':
                    #raise Exception('No Answer')
                    pass
                else:
                    score = float(torch.max(outputs[0]))
                    #print(ans,score)
                    answer.add_ans(ans,score,src,pb)
                    #answer += ans + ', '
                    #print(ans,torch.max(answer_start_scores),torch.max(answer_end_scores))
                    #break
        else:
            outputs = self.model(**self.inputs)

            #print(outputs[0])

            answer_start = torch.argmax(outputs[0])  # get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(outputs[1]) + 1  # get the most likely end of answer with the argmax of the score

            ans = self.convert_ids_to_string(self.inputs['input_ids'][0][
                                              answer_start:answer_end])

            if ans in ['',' ','  ']:
              pass
            else:
              score = float(torch.max(outputs[0]))
              #print(ans,score)
              answer.add_ans(ans,score,src,pb)
        #if len(answer) == 0:
        #  raise Error("No Answer")
        return answer

    def convert_ids_to_string(self, input_ids):
        return self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(input_ids))

In [6]:
G_TEXT="""
함평 대한민국 국향대전 시작 광주전남 축제 본격 개최
코로나19에 지친 시민들 만개한 형형색색 국화보며 힐링
지난해 취소됐던 충장축제도 개최 대면과 비대면 병행 개최
11월 전남 해남미남축제, 순천푸드앤아트페스티벌 등 4개 축제 개최
관람객들이 5일 대한민국국향대전이 열린 전남 함평 엑스포공원에서 국화를 감상하고 있다. 김한영 기자
관람객들이 5일 대한민국국향대전이 열린 전남 함평 엑스포공원에서 국화를 감상하고 있다. 김한영 기자
위드 코로나 정책이 본격 시행되고 있는 가운데 5일 함평에서 막을 연 대한민국 국향대전을 시작으로 광주전남에서는 다양한 축제가 펼쳐진다.
대한민국 국향대전이 5일 오후 함평 엑스포공원.
국향대전 매표소 앞엔 줄이 늘어섰고, 입장을 기다리며 들뜬 관객들로 북적였다.
엑스포공원에는 지구볼 손 모형, 바다거북과 물고기, 꿈나무 희망나무 등 조형물부터 600여 점의 야간경관 조형물도 설치돼 남녀노소 모두에게 즐거운 볼거리를 제공하고 있다.
국화의 매력을 작품으로 살펴보는 분재작품 전시관과 대한민국 국화동호회 작품관을 비롯해 관람객을 빛의 세계로 안내할 미디어아트 전시도 준비돼 있었다.
관람객들이 5일 대한민국국향대전이 열린 전남 함평 엑스포공원에서 국화 분재를 감상하고 있다. 김한영 기자
관람객들이 5일 대한민국국향대전이 열린 전남 함평 엑스포공원에서 국화 분재를 감상하고 있다. 김한영 기자
코로나에 지쳤던 관람객들은 만개한 형형색색 국화를 보며 힐링의 시간을 가졌다. 관람객들은 국화로 만든 조형물 앞에서 사진도 찍고 전시관에 전시 중인 국화 분재도 구경했다.
서울에서 온 박영숙(61·여)씨는 "국화하면 함평이라서 서울에서 찾아왔다"며 "가을 분위기가 나서 너무 좋다"고 말했다. 광주에 사는 윤인선(31·여)씨도 "예쁜 꽃을 보니까 힐링되는 기분이 든다"며 "국화뿐만 아니라 다양한 조형물도 볼 수 있어서 좋았다"고 말했다.
관람객들은 진한 국화 향을 맡으며 깊어가는 가을 정취를 만끽했다.
함평군은 코로나19 여파로 지난해 취소됐던 국향대전이 오랜만에 관람객을 맞이하는 만큼 '안심‧안전축제'로 거듭나기 위해 출입구에 전신 소독기를 비치하는 방역에 만전을 기했다.
지난해 코로나19 여파로 취소됐던 광주 충장축제도 오는 18일부터 21일까지 사흘 동안 열린다.
광주 동구청은 코로나 일상시대에 발맞춰 주민과 방문객들의 안전을 최우선으로 대면과 비대면을 병행해 '제18회 추억의 충장축제'를 개최한다.  대면 프로그램은 5·18민주광장, 신서석로, 국립아시아문화전당(ACC) 부설 주차장, 용산체육공원, 전일빌딩245, 충장로·금남로 지하상가 등 동구 일원에서 진행된다.
충장축제 5대 시민참여형 프로그램인 △충장 텐트 공연예술제 △드라이브 인 추억 콘서트 △추억의 소울푸드 △필름 카메라 IN 광주여행 △우리 마을 골목에서 즐기는 충장축제를 통해 방문객들에게 특별한 추억을 선사할 예정이다.
동구청은 개별적 방역관리가 어려운 먹거리·체험부스는 배제하는 대신 대면접촉을 최소화할 수 있는 포토존과 전시물을 확대할 방침이다.
11월 전남에서는 대한민국 국향대전을 시작으로 해남미남축제(12~14일)와 순천푸드앤아트페스티벌(12~13일), 함평 대한민국난명품 대제전(27~28일) 등 4개 축제가 대면행사로 치뤄질 예정이다.
"""

In [36]:
import sys
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import wikipedia as wiki
import pprint as pp
from collections import OrderedDict
import scipy
import requests
from bs4 import BeautifulSoup

class Korean_QA_on_Wiki:
  def __init__(self, document_reader,sentence_embedder):
    self.reader = document_reader
    self.embedder = sentence_embedder
    wiki.set_lang('ko')

  def __search_from_wiki(self,question,max_rank):
    results = wiki.search(question,results=max_rank)
    contents = []
    for result in results:
      try:
        page = wiki.page(result)
        #print(f"Top wiki result: {page}")
        text = page.content
        contents.append((text,page))
      except Exception as ex:
        print(ex)
    return contents

  def __search_from_naver(self,question,max_rank):
    contents = []
    url = 'https://search.naver.com/search.naver'
    for w in ['view','kin','news','kdic']:
      params = {'query': question,'where': w,}
      response = requests.get(url, params=params)
      html = response.text
      #뷰티풀소프의 인자값 지정
      soup = BeautifulSoup(html, 'html.parser')
      #쪼개기
      #title_list = soup.find_all('a', href=True)
      title_list = soup.select('.api_txt_lines')
      #print(title_list)
      tmp = []
      for tag in title_list:
        tmp.append(tag.text)
      contents.append((''.join(tmp),url + '?where='+w))
      tmp.clear()
    #print(contents)
    return contents



  def question(self, questions, max_rank = 5):
    answers = {}
    for question in questions:
        print(f"Question: {question}")

        contents = []
        contents.extend(self.__search_from_naver(question,max_rank))
        contents.extend(self.__search_from_wiki(question,max_rank))
        #contents.append((G_TEXT,"page"))
        src_count = len(contents)
        pb = ProgressBar(total=src_count+1,prefix='Searching answers...')
        pb.printProgress(0,str(src_count)+' of sources')
        #print('-- Source count : ', len(contents))
        answer = AnswerVoter()
        if self.embedder is None:
            for context, src in contents:
                #text = contents[idx][0]
                pb.printProgress(+1,src)
                #print('-- source :', contents[idx][1])
                try:
                    self.reader.tokenize(question, context)
                    self.reader.get_answer(answer,src,pb)
                    #answer.add_src(contents[idx][1])
                    #t = (self.reader.get_answer(answer),contents[idx][1])
                    #print(f"Answer: {t[0]}", f" from {t[1]}")
                    #answer_list.append(t)
                except Exception as ex:
                    #pb.printProgress(0,sys.exc_info()[0])
                    print(ex,sys.exc_info())
                    #pass
        else:
            corpus_embeddings = self.embedder.encode([a for (a,b) in contents],show_progress_bar=False)
            query_embeddings = self.embedder.encode([question])
            distances = scipy.spatial.distance.cdist(query_embeddings, corpus_embeddings, "cosine")[0]

            results = zip(range(len(distances)), distances)
            results = sorted(results, key=lambda x: x[1])


            for idx, distance in results:
                context = contents[idx][0]
                pb.printProgress(+1,contents[idx][1])
                #print('-- source :', contents[idx][1])
                try:
                    self.reader.tokenize(question, context)
                    self.reader.get_answer(answer,contents[idx][1],pb)
                    #answer.add_src(contents[idx][1])
                    #t = (self.reader.get_answer(answer),contents[idx][1])
                    #print(f"Answer: {t[0]}", f" from {t[1]}")
                    #answer_list.append(t)

                except Exception as ex:
                    #pb.printProgress(0,sys.exc_info()[0])
                    print(ex,sys.exc_info())
                    #pass
        answers[question] = answer.get_ans()
        pb.printProgress(+1,"완료")
        answer.print()
        print(' ')
    return answers


In [37]:
kqaw = Korean_QA_on_Wiki(DocumentReader(tokenizer,model), embedder)

In [39]:
answers = kqaw.question(["베트남에서 가장 인기있는 관광지는 어디인가요?"])

Question: 베트남에서 가장 인기있는 관광지는 어디인가요?
Searching answers... |████████████████████| 100.0%   완료
Answer: 호치민시와 하노이  score: 11.120749473571777  source: ['https://search.naver.com/search.naver?where=view']
Answer: 다낭  score: 10.792304039001465  source: ['https://search.naver.com/search.naver?where=news']
Answer: 냐짱  score: 9.128152847290039  source: [<WikipediaPage '냐짱'>]
Answer: 축구  score: 8.475028038024902  source: [<WikipediaPage '태국의 문화'>]
Answer: 호치민  score: 6.090912818908691  source: ['https://search.naver.com/search.naver?where=kin']
 


In [40]:
answers = kqaw.question(["베트남 다낭에서 맛집 한국식당 이름은?"])

Question: 베트남 다낭에서 맛집 한국식당 이름은?
Searching answers... |████████████████████| 100.0%   완료
 


In [41]:
answers = kqaw.question(["코로나19는 언제 종식되는가?"])

Question: 코로나19는 언제 종식되는가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 6월 15일  score: 9.974505424499512  source: [<WikipediaPage '뉴질랜드의 코로나19 범유행'>]
 


In [42]:
answers = kqaw.question(["아시아나IDT의 대표이사는 누구인가?"])

Question: 아시아나IDT의 대표이사는 누구인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 박찬법  score: 11.626656532287598  source: ['https://search.naver.com/search.naver?where=kin']
Answer: 박세창  score: 11.45822525024414  source: ['https://search.naver.com/search.naver?where=view']
Answer: 한창수  score: 11.200672149658203  source: ['https://search.naver.com/search.naver?where=news']
Answer: 금호고속, 금호산업  score: 3.114767551422119  source: ['https://search.naver.com/search.naver?where=kdic']
 


In [43]:
answers = kqaw.question(["아시아나항공은 어디에 매각되는가?"])

Question: 아시아나항공은 어디에 매각되는가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 금호산업  score: 9.347007751464844  source: ['https://search.naver.com/search.naver?where=kdic']
Answer: 대한항공  score: 7.730260372161865  source: ['https://search.naver.com/search.naver?where=news']
Answer: 금호아시아나그룹  score: 5.6735758781433105  source: ['https://search.naver.com/search.naver?where=kin']
 


In [44]:
answers = kqaw.question(["북한에서 실질적인 권력자는 누구인가?",
                           "세계에서 가장 넓은 호수는?",
                           "오로라가 가장 잘 보이는 곳은?",
                           "심장이 죄어오듯이 아프면 의심되는 병은 무엇인가?",
                           "항문에서 피가 나는 병은 무엇인가?",
                           "김재규는 박정희를 왜 죽였는가?",
                           "케네디를 죽인 암살범은 누구인가?",
                           "술 취하지 않는 방법은?",
                           "사람을 사랑해서 생기는 병은?",
                           "부모는 자식을 왜 사랑하는가?",
                           "나의 와이프는 나를 사랑하는가?",
                           "신은 존재 하는가?",
                           "사람의 인생에서 가장 소중한 것은 무엇인가?",
                           "바람난 여자는 다시 돌아올 수 있는가?",
                           "위가 쓰리고 아플 때 어떤 약을 복용해야 하는가?",
                           "눈알이 빠지면 어떻게 되는가?"])

Question: 북한에서 실질적인 권력자는 누구인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 총비서  score: 6.773006439208984  source: ['https://search.naver.com/search.naver?where=news']
 
Question: 세계에서 가장 넓은 호수는?
Searching answers... |████████████████████| 100.0%   완료
Answer: 카스피해  score: 21.056517601013184  source: ['https://search.naver.com/search.naver?where=kin', <WikipediaPage '호수'>]
Answer: 빅토리아호  score: 19.68979263305664  source: [<WikipediaPage '빅토리아호'>, <WikipediaPage '아프리카 대호수'>]
Answer: 바이칼 호수  score: 8.188878059387207  source: ['https://search.naver.com/search.naver?where=view']
Answer: 바이칼호  score: 7.776245594024658  source: [<WikipediaPage '바이칼호'>]
Answer: 티베트 자치구 남초호  score: 6.32595157623291  source: [<WikipediaPage '염호'>]
 
Question: 오로라가 가장 잘 보이는 곳은?
Searching answers... |████████████████████| 100.0%   완료
Answer: 국제우주정거장  score: 9.926204681396484  source: ['https://search.naver.com/search.naver?where=news']
Answer: 남극및 북극 양극지방  score: 8.935654640197754  source: [<Wik

In [45]:
answers = kqaw.question(["아시아나항공은 어디에 매각될 것인가?",
                         "박세창은 어느 회사의 사장인가?"])

Question: 아시아나항공은 어디에 매각될 것인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 금호산업  score: 8.035517692565918  source: ['https://search.naver.com/search.naver?where=kdic']
 
Question: 박세창은 어느 회사의 사장인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 아시아나IDT  score: 18.40152931213379  source: ['https://search.naver.com/search.naver?where=news', 'https://search.naver.com/search.naver?where=view']
Answer: 삼화교통  score: 9.245410919189453  source: [<WikipediaPage '박인천'>]
 


In [46]:
answers = kqaw.question(["아시아나항공 사장의 이름은?",
                         "금호건설의 사장은 누구인가?"])

Question: 아시아나항공 사장의 이름은?
Searching answers... |████████████████████| 100.0%   완료
Answer: 전펑  score: 10.870772361755371  source: ['https://search.naver.com/search.naver?where=kdic']
Answer: 조원태  score: 8.332274436950684  source: [<WikipediaPage '조원태'>]
 
Question: 금호건설의 사장은 누구인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 박세창  score: 10.947713851928711  source: ['https://search.naver.com/search.naver?where=view']
Answer: △서재환  score: 8.246779441833496  source: ['https://search.naver.com/search.naver?where=news']
Answer: 대우건설인수, 효성의 하이닉스 인수실패등등. 많은 사례에서 채권단이 크게 배울만큼 배웠고 생각보다는 현대그룹도... 포스코건설, 대림, sk건설, 롯데건설, 금호건설, 태영건설, 계룡건설 연봉 복지 좋은곳부터 순서대로 나열좀해줘요... 그래서 중소기업 사장이 힘든 거에요. 대기업 구매팀 젊은 대리 과장한테 굽신굽신 거려야 됩니다. X같아요. 공무원 갑질이 일어나는 구조적... 다 비슷한데 누굴 줘요. 만만하고 착하고 작년에 승진한 사람 그냥 주는 겁니다. 그리고 내년에 SA줘서 연봉 상승률 복구되게 해주는 거죠.... 인하대 교과면접. 말그대로 인하대 교과면접말인데요... 제가 사회인프라공학과를 지원했는데 교과면접에 학생부반영되나요?? 교과는아니다고들었는데 진로선생님께서 들어가신다고하셔서,,, 그리고 면접 준비법도좀 알려주세요,,,... 1 - 1 ] 건설분야 CAE 소프트웨어 관련 KCC건설 추천 

In [47]:
# 한진중공업 매각 우선협상대상자
answers = kqaw.question(["한진중공업 매각 우선협상대상자는 어디인가?"])

Question: 한진중공업 매각 우선협상대상자는 어디인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 동부건설 컨소시엄  score: 11.790081977844238  source: ['https://search.naver.com/search.naver?where=news']
Answer: JKL파트너스  score: 10.622962951660156  source: ['https://search.naver.com/search.naver?where=kin']
Answer: 동부건설 컨소시엄이 우선협상대상자로 선정됐다. 한진중공업은 1937년 설립된 최초의 조선소이자, 그동안 대한민국... 한진중공업 매각, 우선협상대상자... 동부건설 선정온갖 소문이 난무했던 한진중공업 매각은 최종적으로 동부건설  score: 5.739238262176514  source: ['https://search.naver.com/search.naver?where=view']
 


In [48]:
answers = kqaw.question(["아파트 값은 계속 오를 것인가?",
                         "코로나는 언제 종식 될 것인가?"])

Question: 아파트 값은 계속 오를 것인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 해모로  score: 6.334014415740967  source: [<WikipediaPage 'HJ중공업'>]
 
Question: 코로나는 언제 종식 될 것인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 6월 15일  score: 9.592249870300293  source: [<WikipediaPage '뉴질랜드의 코로나19 범유행'>]
 


In [49]:
answers = kqaw.question(["단백질의 화학식 구성은 어떻게 되는가?"])

Question: 단백질의 화학식 구성은 어떻게 되는가?
Searching answers... |████████████████████| 100.0%   완료
Answer: NH2CHRnCOOH ) n  score: 9.902149200439453  source: [<WikipediaPage '단백질'>]
Answer: 알라닌  score: 3.8399808406829834  source: [<WikipediaPage '아미노산'>]
 


In [50]:
answers = kqaw.question(["우리나라 특허의 권리보장 기간은 몇년인가?"])

Question: 우리나라 특허의 권리보장 기간은 몇년인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 20년  score: 13.85037636756897  source: ['https://search.naver.com/search.naver?where=kdic', 'https://search.naver.com/search.naver?where=view']
Answer: 2 ~ 3년  score: 4.400360584259033  source: ['https://search.naver.com/search.naver?where=news']
 


In [51]:
answers = kqaw.question(["발열 마른기침 피로감 등의 증상을 보이면 어떤 병이 의심되는가?"])

Question: 발열 마른기침 피로감 등의 증상을 보이면 어떤 병이 의심되는가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 심장질환  score: 10.331414222717285  source: ['https://search.naver.com/search.naver?where=news']
Answer: 광견병  score: 8.204215049743652  source: ['https://search.naver.com/search.naver?where=kdic']
Answer: 기침  score: 4.397464275360107  source: [<WikipediaPage '코로나19 범유행'>]
 


In [53]:
answers = kqaw.question(["흉부통증과 호흡곤란, 쉰목소리, 가끔 피가 썩인 가래도 있습니다. 어떤 병일까요?"])

Question: 흉부통증과 호흡곤란, 쉰목소리, 가끔 피가 썩인 가래도 있습니다. 어떤 병일까요?
Searching answers... |████████████████████| 100.0%   완료
 


In [54]:
answers = kqaw.question(["똥을 싸고 나면 휴지에 피가 뭍습니다. 의심되는 병은 무엇인가요?"])

Question: 똥을 싸고 나면 휴지에 피가 뭍습니다. 의심되는 병은 무엇인가요?
Searching answers... |████████████████████| 100.0%   완료
Answer: 변비  score: 8.310355186462402  source: ['https://search.naver.com/search.naver?where=view']
Answer: 치질  score: 6.736917972564697  source: ['https://search.naver.com/search.naver?where=kin']
 


In [55]:
answers = kqaw.question(["신경망 알고리즘의 활성화 함수에는 어떤 것이 있나요?"])

Question: 신경망 알고리즘의 활성화 함수에는 어떤 것이 있나요?
Searching answers... |████████████████████| 100.0%   완료
Answer: 입력 이미지의 픽셀  score: 10.152823448181152  source: [<WikipediaPage '인공 신경망'>]
Answer: 뉴런 가중치  score: 8.611798286437988  source: ['https://search.naver.com/search.naver?where=news']
Answer: 도함수  score: 8.3803071975708  source: [<WikipediaPage '기울기 소멸 문제'>]
Answer: 뉴런  score: 7.798391819000244  source: [<WikipediaPage '퍼셉트론'>]
Answer: 미분  score: 3.88582444190979  source: ['https://search.naver.com/search.naver?where=kin']
 


In [56]:
answers = kqaw.question(["현존하는 인공지능 중 가장 성능이 우수한 것은 무엇입니까?"])

Question: 현존하는 인공지능 중 가장 성능이 우수한 것은 무엇입니까?
Searching answers... |████████████████████| 100.0%   완료
Answer: 칩셋  score: 10.755560874938965  source: ['https://search.naver.com/search.naver?where=view']
Answer: 대화 언어 모델  score: 9.125129699707031  source: ['https://search.naver.com/search.naver?where=news']
 


In [57]:
answers = kqaw.question(["이세돌을 이긴 것은 무엇입니까?"])

Question: 이세돌을 이긴 것은 무엇입니까?
Searching answers... |████████████████████| 100.0%   완료
Answer: 알파고  score: 8.71656322479248  source: ['https://search.naver.com/search.naver?where=view', 'https://search.naver.com/search.naver?where=kin']
 


In [58]:
answers = kqaw.question(["피타고라스는 어느 나라 사람인가?"])

Question: 피타고라스는 어느 나라 사람인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 그리스  score: 20.505011558532715  source: ['https://search.naver.com/search.naver?where=kin', 'https://search.naver.com/search.naver?where=view', 'https://search.naver.com/search.naver?where=news']
 


In [59]:
answers = kqaw.question(["아이즈원 멤버 인원수는?"])

Question: 아이즈원 멤버 인원수는?
Searching answers... |████████████████████| 100.0%   완료
Answer: 12명  score: 11.783061027526855  source: ['https://search.naver.com/search.naver?where=news']
Answer: 6명  score: 11.573765754699707  source: ['https://search.naver.com/search.naver?where=view']
Answer: 8명  score: 8.347689628601074  source: ['https://search.naver.com/search.naver?where=kin']
 


In [60]:
answers = kqaw.question(["트와이스 중에 가장 인기 있는 사람은?"])

Question: 트와이스 중에 가장 인기 있는 사람은?
Searching answers... |████████████████████| 100.0%   완료
Answer: 박진영  score: 9.681979179382324  source: [<WikipediaPage '박진영'>]
Answer: 나연  score: 9.32990550994873  source: [<WikipediaPage '나연'>]
Answer: 가는 세월  score: 8.899544715881348  source: ['https://search.naver.com/search.naver?where=kdic']
Answer: 뉴진스  score: 8.232033729553223  source: ['https://search.naver.com/search.naver?where=news']
Answer: 사나  score: 6.566624164581299  source: [<WikipediaPage '사나 (일본의 가수)'>]
 


In [61]:
answers = kqaw.question(["벤츠 자동차가 처음 발명된 년도는?"])

Question: 벤츠 자동차가 처음 발명된 년도는?
Searching answers... |████████████████████| 100.0%   완료
Answer: 1886년  score: 11.372413635253906  source: [<WikipediaPage '만하임'>]
Answer: 1883년  score: 10.976832389831543  source: ['https://search.naver.com/search.naver?where=kin']
Answer: 1885년  score: 10.761576652526855  source: [<WikipediaPage '자동차'>]
Answer: 2008년  score: 9.288729667663574  source: ['https://search.naver.com/search.naver?where=news']
 


In [62]:
answers = kqaw.question(["교통사고 대비를 위해 들어야 하는 보험은 무엇인가?"])

Question: 교통사고 대비를 위해 들어야 하는 보험은 무엇인가?
Searching answers... |████████████████████| 100.0%   완료
Answer: 개인보험 개인보험  score: 10.217618942260742  source: ['https://search.naver.com/search.naver?where=kdic']
Answer: 후유 장애 보험  score: 7.257519245147705  source: ['https://search.naver.com/search.naver?where=news']
Answer: 운전자보험  score: 4.809130668640137  source: ['https://search.naver.com/search.naver?where=kin']
Answer: 운전자보험교통사고위로금  score: 4.487279891967773  source: ['https://search.naver.com/search.naver?where=view']
 


In [63]:
answers = kqaw.question(["너의 이름은 무엇이니?"])

Question: 너의 이름은 무엇이니?
Searching answers... |████████████████████| 100.0%   완료
Answer: 식물의 이름  score: 11.028470039367676  source: ['https://search.naver.com/search.naver?where=news']
Answer: 고등학생  score: 4.949797630310059  source: [<WikipediaPage '물음표'>]
 


In [64]:
answers = kqaw.question(["너는 남자니 여자니?"])

Question: 너는 남자니 여자니?
Searching answers... |████████████████████| 100.0%   완료
 


#KorQuad2.0 학습.

https://github.com/huggingface/datasets

https://huggingface.co/transformers/custom_datasets.html#qa-squad



In [35]:

if True:
    from google.colab import drive
    drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!pip install datasets

In [ ]:
from datasets import list_datasets, load_dataset, list_metrics, load_metric, load_from_disk

# Print all the available datasets
print(list_datasets())

# Load a dataset and print the first example in the training set
squad_dataset = load_dataset('squad_kor_v2',cache_dir ='/content/drive/MyDrive/korQuAD2.1/dataset')


In [ ]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('/content/drive/MyDrive/korQuAD2.1/dataset/squad_kor_v2/squad_kor_v2/2.1.0/8e4ee4e5757761cf13f00b2d4e4cef2e842c0ea3c57050fec9fafc8fec60e128/squad_kor_v2-train.arrow')
val_contexts, val_questions, val_answers = read_squad('/content/drive/MyDrive/korQuAD2.1/dataset/squad_kor_v2/squad_kor_v2/2.1.0/8e4ee4e5757761cf13f00b2d4e4cef2e842c0ea3c57050fec9fafc8fec60e128/squad_kor_v2-validation.arrow')

In [ ]:
squad_dataset['train']

In [ ]:
!pip install transformers

In [ ]:
train_contexts = squad_dataset['train']['context']
train_questions = squad_dataset['train']['question']
train_answers = squad_dataset['train']['answer']

In [ ]:
val_contexts = squad_dataset['validation']['context']
val_questions = squad_dataset['validation']['question']
val_answers = squad_dataset['validation']['answer']

In [ ]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [ ]:
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained('monologg/kobert')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)